# Librerias


In [ ]:
import numpy as np
import unicodedata
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random

# --- 1. Cargar y limpiar texto ---
Primero abrí el archivo de poemas y realicé una limpieza básica pero muy importante: pasé todo a minúsculas, eliminé tildes sin quitar la letra base, borré todos los números y también quité símbolos raros o especiales que no aportan nada al modelo. Al final, normalicé los espacios para que el texto quedara uniforme y sin ruido innecesario.

In [ ]:
with open("/content/Pablo Neruda 363 Spanish poems - Dataset .txt", "r", encoding="utf-8") as file:
    text = file.read()

def clean_text(txt):
    txt = txt.lower()
    txt = ''.join(
        c for c in unicodedata.normalize('NFD', txt)
        if unicodedata.category(c) != 'Mn'
    )
    txt = re.sub(r"\d+", "", txt)               # Elimina todos los números
    txt = re.sub(r"[^a-zñ\s]", "", txt)         # Elimina cualquier otro caracter raro
    txt = re.sub(r"\s+", " ", txt).strip()      # Normaliza espacios
    return txt

cleaned_text = clean_text(text)

# --- 2. Tokenización a nivel palabra ---
Usé Tokenizer de Keras para convertir cada palabra del corpus en un número entero único. Esto es fundamental para que el modelo pueda trabajar con datos numéricos. También obtuve el tamaño total del vocabulario, que es útil para definir la red neuronal más adelante.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
tokens = tokenizer.texts_to_sequences([cleaned_text])[0]
vocab_size = len(tokenizer.word_index) + 1

# --- 3. Generador de secuencias ---
Aquí definí una clase que construye secuencias de entrenamiento dinámicamente por lotes. Esto ayuda a no cargar todo en RAM al mismo tiempo, que era el problema que tenía antes. El generador produce pares de entrada y salida, donde la entrada es una secuencia de palabras y la salida es la siguiente palabra esperada

In [ ]:
class NerudaSequenceGenerator(Sequence):
    def __init__(self, tokens, seq_length, batch_size, vocab_size):
        self.tokens = tokens
        self.seq_length = seq_length
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.indexes = range(seq_length, len(tokens))

    def __len__(self):
        return int(np.ceil((len(self.tokens) - self.seq_length) / self.batch_size))

    def __getitem__(self, idx):
        X, y = [], []
        start = idx * self.batch_size + self.seq_length
        end = min(start + self.batch_size, len(self.tokens))

        for i in range(start, end):
            seq = self.tokens[i - self.seq_length:i]
            X.append(seq[:-1])
            y.append(seq[-1])

        X = pad_sequences(X, maxlen=self.seq_length-1, padding='pre')
        y = np.array(y)  # Sparse output (no one-hot)
        return X, y

# --- 4. Crear generador ---
Aquí inicializo el generador con secuencias de longitud 20 y un batch size de 64, que me pareció un buen punto medio entre rendimiento y eficiencia de memoria.

In [ ]:
sequence_length = 20
batch_size = 64
generator = NerudaSequenceGenerator(tokens, sequence_length, batch_size, vocab_size)

# --- 5. Crear modelo ---
Diseñé un modelo secuencial con una capa de embeddings (para representar semánticamente las palabras), seguida de dos capas LSTM con Dropout entre ellas para evitar overfitting, y finalmente una capa Dense que predice la siguiente palabra. Usé sparse_categorical_crossentropy como función de pérdida para no tener que usar one-hot encoding y ahorrar memoria.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- 6. Entrenamiento ---
Finalmente, entrené el modelo usando el generador. Le puse EarlyStopping por si el modelo deja de mejorar, y ModelCheckpoint para guardar el mejor modelo encontrado. Lo dejé entrenando por 100 épocas porque quiero un resultado de calidad, aunque sé que puedo detenerlo antes si la pérdida se estabiliza.

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=5)
checkpoint = ModelCheckpoint("modelo_poemas.h5", save_best_only=True)

model.fit(generator, epochs=100, callbacks=[early_stop, checkpoint])


# Prueba
Esta función toma una frase inicial (seed_text) y genera un número determinado de palabras (next_words). La temperatura ajusta la aleatoriedad del modelo: si es baja, el modelo elige las palabras más probables; si es alta, permite mayor variación y creatividad

In [ ]:
def generate_poem(seed_text, next_words=30, temperature=1.0):
    result = []
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=sequence_length-1, padding='pre')

        # Obtener predicciones
        preds = model.predict(token_list, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')

        # Aplicar temperatura
        preds = np.log(preds + 1e-7) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        # Elegir palabra siguiente por muestreo probabilístico
        predicted_index = np.random.choice(range(len(preds)), p=preds)

        # Convertir índice a palabra
        output_word = tokenizer.index_word.get(predicted_index, '')
        if output_word == '':
            continue  # Si no se encuentra, salta esa predicción

        seed_text += ' ' + output_word
        result.append(output_word)
    return ' '.join(result)


# Resultado

In [ ]:
seed = "amor"
print(generate_poem(seed, next_words=50, temperature=0.2))

# Conclusiones
Este proyecto me permitió entrenar una red neuronal recurrente (LSTM) capaz de generar poesía en español a partir de un corpus extenso de Pablo Neruda. Durante el desarrollo, enfrenté algunos retos de memoria debido a la cantidad de secuencias generadas, pero implementé un generador de datos personalizado que resolvió eficazmente ese problema, permitiendo entrenar el modelo por lotes y sin saturar la RAM.

Gracias a la limpieza adecuada del texto (removiendo tildes, símbolos y números), el modelo logró concentrarse en las palabras realmente relevantes. El uso de embeddings ayudó a capturar relaciones semánticas entre palabras, y la arquitectura LSTM con Dropout previno el sobreajuste.

Finalmente, implementé una función de generación de texto controlada por temperatura, lo que me permitió ajustar la creatividad de los poemas generados. El modelo logró aprender estilos coherentes y producir versos que, si bien no son perfectos, tienen una estructura y vocabulario similar al original, lo cual valida el entrenamiento.